In [340]:
import pandas as pd
import requests
from pytrends.request import TrendReq
import pygsheets
from bs4 import BeautifulSoup as bs
import pprint


In [235]:
gc = pygsheets.authorize(outh_file="D:\Documents\Projects\FIFA\client_secret.json")

In [455]:
kw_list = ["David De Gea","Manuel Neuer","Marc-Andre ter Stegen","Alisson","Keylor Navas"]
pytrend=TrendReq()
pytrend.build_payload(kw_list)
interest_over_time_df=pytrend.interest_over_time()
ts=pd.to_datetime(interest_over_time_df.index)
interest_over_time_df['date']=ts.strftime("%m.%d.%Y")
interest_over_time_df = interest_over_time_df.loc[interest_over_time_df['isPartial'] == 'False']
interest_over_time_df = interest_over_time_df[["date","David De Gea","Manuel Neuer","Marc-Andre ter Stegen","Alisson","Keylor Navas"]]

In [456]:
print(interest_over_time_df)

                  date  David De Gea  Manuel Neuer  Marc-Andre ter Stegen  \
date                                                                        
2013-06-09  06.09.2013             2             2                      0   
2013-06-16  06.16.2013             3             2                      0   
2013-06-23  06.23.2013             1             3                      0   
2013-06-30  06.30.2013             1             1                      0   
2013-07-07  07.07.2013             1             1                      0   
2013-07-14  07.14.2013             1             1                      0   
2013-07-21  07.21.2013             1             2                      0   
2013-07-28  07.28.2013             1             2                      0   
2013-08-04  08.04.2013             1             2                      0   
2013-08-11  08.11.2013             1             2                      0   
2013-08-18  08.18.2013             1             1                      0   

In [238]:
interest_over_time_df2 = pd.melt(interest_over_time_df,id_vars=['date'], var_name=
'goalie_name',value_name='index')
print(interest_over_time_df2)

            date   goalie_name  index
0     06.09.2013  David De Gea      2
1     06.16.2013  David De Gea      3
2     06.23.2013  David De Gea      1
3     06.30.2013  David De Gea      1
4     07.07.2013  David De Gea      1
5     07.14.2013  David De Gea      1
6     07.21.2013  David De Gea      1
7     07.28.2013  David De Gea      1
8     08.04.2013  David De Gea      1
9     08.11.2013  David De Gea      1
10    08.18.2013  David De Gea      1
11    08.25.2013  David De Gea      1
12    09.01.2013  David De Gea      1
13    09.08.2013  David De Gea      1
14    09.15.2013  David De Gea      1
15    09.22.2013  David De Gea      1
16    09.29.2013  David De Gea      1
17    10.06.2013  David De Gea      1
18    10.13.2013  David De Gea      1
19    10.20.2013  David De Gea      1
20    10.27.2013  David De Gea      1
21    11.03.2013  David De Gea      1
22    11.10.2013  David De Gea      1
23    11.17.2013  David De Gea      1
24    11.24.2013  David De Gea      1
25    12.01.

In [239]:
sh=gc.open('Top FIFA Goalies')
wks = sh.worksheet_by_title("google_trends_data")

wks.set_dataframe(interest_over_time_df2,(1,1))

In [447]:
url='http://www.transfermarkt.com/detailsuche/spielerdetail/suche/9069756'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}
player_info = [[] for i in range(2)]
response=requests.get(url,headers=headers)
#response.status_code
soup = bs(response.content,"html.parser")
table = soup.find(class_='items')
for item in table.find_all('a',{'class':'spielprofil_tooltip'}):
    #print(item.get('title') + ':' + item.get('id'))
    player_info[0].append(item.get('title').replace(' ','-'))
    player_info[1].append(item.get('id'))

In [398]:
print(player_info[0])

['Neymar', 'Lionel-Messi', 'Mohamed-Salah', 'Harry-Kane', 'Kylian-Mbappé', 'Eden-Hazard', 'Paulo-Dybala', 'Antoine-Griezmann', 'Philippe-Coutinho', 'Cristiano-Ronaldo', 'Leroy-Sané', 'Raheem-Sterling', 'Romelu-Lukaku', 'Gareth-Bale', 'Robert-Lewandowski', 'Gabriel-Jesus', 'Ousmane-Dembélé', 'Roberto-Firmino', 'Sergio-Agüero', 'Marco-Asensio', 'Mauro-Icardi', 'Pierre-Emerick-Aubameyang', 'Sadio-Mané', 'Luis-Suárez', 'Alexis-Sánchez']


In [448]:
player_response = requests.get("https://www.transfermarkt.com/"+ str(player_info[0][0])+ "/leistungsdatendetails/spieler/" + str(player_info[1][0]) + "/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1", headers = headers )
player_soup = bs(player_response.content,"html.parser")
print(player_response.status_code)

200


In [445]:

player_response = requests.get("https://www.transfermarkt.com/"+ str(player_info[0][0])+ "/leistungsdatendetails/spieler/" + str(player_info[1][0]) + "/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1", headers = headers )
player_soup = bs(player_response.content,"html.parser")

player_headers = []
table = player_soup.find(class_='items')
for item in thead.find_all('a',{'class':'sort-link'}):
    #print(item.contents[0])
    if (str(type(item.contents[0])) == "<class 'bs4.element.NavigableString'>"):
        player_headers.append(item.contents[0])
    else:
        player_headers.append(item.contents[0].get('title'))

table_body = table.find('tbody')
rows = table_body.find_all('tr')

data = [[] for i in range(len(player_headers))]

for row in rows:
    i=0
    for element in row.find_all('td'):
        if (str(element.contents[0])[0]) == "<":
            if str(type(element.find('a'))) == "<class 'NoneType'>":
                1+1
            else:
                if str(element.find('a').contents[0])[0]=="<":
                    
                    data[i].append(str(element.find('a').get('href')).strip('/').split('/')[0])
                else:
                    if element.find('a').contents[0] == "-":
                        data[i].append(0)
                    else:
                        data[i].append(element.find('a').contents[0])
        else:
            if element.find('a').contents[0] == "-":
                data[i].append(0)
            else:
                data[i].append(element.contents[0])
        i=i+1
        
data.pop(1)
player_headers.pop(2)
player_df = pd.DataFrame(data)
player_df = player_df.transpose()
player_df.columns = player_headers
player_df



AttributeError: 'int' object has no attribute 'items'

In [451]:
frames={}
player_headers = ["Player"]

data = [[] for i in range(19)]
head=0
for x in range(len(player_info[0])):
    #player_url= "https://www.transfermarkt.com/"+ str(player_info[0][x])+ "/leistungsdatendetails/spieler/" + str(player_info[1][x])
    player_response = requests.get("https://www.transfermarkt.com/"+ str(player_info[0][x])+ "/leistungsdatendetails/spieler/" + str(player_info[1][x]) + "/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1", headers = headers )
    player_soup = bs(player_response.content,"html.parser")
    
    table = player_soup.find(class_='items')

    if head == 0:
        for item in thead.find_all('a',{'class':'sort-link'}):
        #print(item.contents[0])
            if str(type(item.contents[0])) == "<class 'bs4.element.NavigableString'>":
                player_headers.append(item.contents[0])
            else:
                player_headers.append(item.contents[0].get('title'))
        head =1
    

    table_body = table.find('tbody')
    rows = table_body.find_all('tr')

    for row in rows:
        i=0
        
        data[i].append(player_info[0][x])
        i=i+1
        for element in row.find_all('td'):
            print(i)
            if (str(element.contents[0])[0]) == "<":
                if str(type(element.find('a'))) == "<class 'NoneType'>":
                    1+1
                else:
                    if str(element.find('a').contents[0])[0]=="<":
                    
                        data[i].append(str(element.find('a').get('href')).strip('/').split('/')[0])
                    else:
                        if str(element.contents[0]) == "-":
                            data[i].append(0)
                        else:
                            data[i].append(element.find('a').contents[0])
            else:
                if str(element.contents[0]) == "-":
                    data[i].append(0)
                else:
                    data[i].append(element.contents[0])
            i=i+1
        
data.pop(2)
player_headers.pop(3)
player_df = pd.DataFrame(data)
player_df = player_df.transpose()
player_df.columns = player_headers
frames[player_info[0][x]]=player_df

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5


In [453]:
#print(frames[player_info[0][0]])
print(player_headers)
#print(player_df)

['Player', 'Season', 'Competition', 'Club(s)', 'Squad', 'Apps', 'PPM', 'Goals', 'Assists', 'own goals', 'Substituted on', 'Substituted off', 'Yellow cards', 'Yellow/red cards', 'Red cards', 'Penalty goals', 'Minutes per goal', 'Minutes played']


In [454]:
#del_worksheet("Top FIFA Strikers")
sh=gc.open('Top FIFA Strikers')

sheet_df = sh.sheet1.get_as_df()

#for x in range(len(sheet_df)):
#    wks = sh.worksheet_by_title(str(sheet_df.iloc[x][0]))
#    sh.del_worksheet(wks)

wks = sh.sheet1
#player_name_df = pd.DataFrame(player_info[0])
wks.set_dataframe(player_df,(1,1))

#for x in range(len(player_info[0])):
#    wks=sh.add_worksheet(player_info[0][x])
#    wks.set_dataframe(frames[player_info[0][x]],(1,1))